In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn import metrics
import json
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet as wn
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
from nltk.corpus import stopwords
import collections
from itertools import groupby
import pickle
import re
import matplotlib.colors as mcolors
import warnings
import requests
import csv

In [2]:
nouns = {x.name().split('.', 1)[0] for x in wn.all_synsets('n')}

### Load the Data

In [3]:
root = "../../data/external/"

# Data Pierre
file_name = root + 'Dataset_Pierre.csv'
df_Pierre = pd.read_csv(file_name, header=[0, 1]) 
df_Pierre = df_Pierre.iloc[: , 1:]
df_Pierre = df_Pierre.set_index(df_Pierre['Species']['species'])

# Data Andrei
file_name = root + 'Dataset_Andrei.csv'
df_Andrei = pd.read_csv(file_name)

# Get Dummies to match DF Pierre
df_Andrei_dummies = pd.get_dummies(df_Andrei.iloc[:, 2:])
# Set species back
df_Andrei_dummies = df_Andrei_dummies.set_index(df_Andrei['Species'])

# Create tuple list for multi index
Andrei_multi_index = []
for top_index in df_Andrei.columns:
    for sub_index in df_Andrei_dummies.columns:
        if top_index in sub_index:

            sub_index = sub_index.split('_')[-1]
            Andrei_multi_index.append((top_index, sub_index))

# Set Mutli index
df_Andrei_dummies.columns = pd.MultiIndex.from_tuples(Andrei_multi_index)

# Data Palms
file_name = root + 'Dataset_Kissling.txt'
df_Daniel = pd.read_csv(file_name,
                 sep='\t', encoding='Latin-1')
palm_species = df_Daniel[~df_Daniel.isnull().any(axis=1)]['SpecName'].values
df_Daniel.set_index('SpecName', inplace=True)

In [4]:
sw = list(stopwords.words('english'))
sw.append('like')
sw.append('color')
sw.append('colour')
sw.append('a')
sw.append('x')

In [5]:
colors = [color[4:] for color in mcolors.TABLEAU_COLORS.keys()]
colors.extend([color for color in mcolors.CSS4_COLORS.keys()])
colors.extend(
    [
        'whitish', 'bluish', 'reddish', 'greenish', 'backish', 'greyish',
        'backish', 'purplish', 'yellowish', 'orangish', 'brownish', 'pinkish'
    ]
)

traits_dict = {
    'Life Form':
    [
        'Tree', 'Shrub', 'Bush', 'Ficus', 'Strangler', 'Liana', 'Parasitic', 'Palm', 'Herbaceous'
    ],
    'Trunk':
    [
        'Trunk', 'Straight', 'Flared', 'Foothills', 'Silt', 'Aerial'
    ],
    'Root':
    [
        'Root', 'Straight', 'Flared', 'Foothills', 'Silt', 'Aerial'
    ],
    'Latex':
    [
        'Latex'
    ],
    'Phyllotaxis': # Leaf Position
    [
        'Phyllotaxis', 'Alternate', 'Whorled', 'Whorls', 'Opposite'
    ],
    'Leaf Composition':
    [
        'Palmate', 'Pinnate', 'Entire', 'Bi-pinnate'
    ],
    'Crown':
    [
        'Crown'
    ],
    'Stem':
    [
        'Stem', 'Circular', 'Square'
    ],
    'Bark':
    [
        'Bark'
    ],
    'Bark Colour':
    [
        'Bark'
    ],
    'Leaf Shape':
    [
        'Simple', 'Bifoliate', 'Trifoliate', 'Digitized', 'Paripinnate', 'Unipinnate', 'Imperipinnate', 
        'Alternate', 'Bipinnate', 'Pinnate', 'Elliptic', 'Elongate', 'Ovate', 'Round', 'Obovate', 'Lanceolate',
        'Kidney-shaped', 'Heart-shaped', 'Spathulate'
    ],
    'Petiole':
    [
        'Petiole', 'Sessile', 'Petiolated', 'Canaliculate', 'Glands', 'Glandular', 
     'Winged' 'Wings', 'Hairs', 'Hair', 'Translucent'
     ],
    'Leaf Colour':
    [
        'Leaf Colour', 'Leaf Color'
    ],
    'Leaf Blade':
    [
        'Leaf Blade', 'Linear', 'Lanceolate', 'Elliptical', 'Obovate', 'Obtriangular', 
        'Obtriangular', 'Asymmetrical', 'Orbicular', 'Bilobed', 'Lobed', 'Lobes', 'Lobe'
    ],
    'Leaf Base':
    [
        'Leaf Base', 'Rounded', 'Cordate', 'Glands'
    ],
    'Leaf Margin':
    [
        'Margin', 'Smooth', 'Wavy', 'Crenate', 'Toothed', 'Teeth', 'Crenate', 'Serrate'
    ],
    'Leaf Apex':
    [
        'Apex', 'Acuminate', 'Apiculate', 'Mucronate', 'Rounded', 'Emarginated'
    ],
    'Leaf side':
    [
        'Glabrous', 'Pubescent', 'Salt Crystals', 'Scales', 'Woolly', 'Powdery'
    ],
    'Leaf glands':
    [
        'Glands', 'Gland', 'Translucent'
    ],
    'Rachis':
    [
        'Rachis', 'Winged'
    ],
    'Vein':
    [
        'Vein'
    ],
    'Tendril':
    [
        'Tendril'
    ],
    'Spine':
    [
        'Spine', 'Prickle', 'Spines', 'Prickles'
    ],
    'Thornes':
    [
        'Thorn', 'Thornes'
    ],
    'Blade Colour':
    [
        'Blade'
    ],
    'Fruit':
    [
        'Drupe', 'Berry', 'Capsule', 'Pod', 'Follicle', 'Achene', 'Winged', 'Follicle',
        'Pod', 'Nutlet', 'Fruit'
    ],
    'Fruit Shape':
    [
        'locular', 'Globose', 'Flattened', 'Elongate', 'Obovoid', 'Ovate', 'Twisted',
        'Curved', 'Pyriform', 'Ovoid'
    ],
    'Fruit Colour':
    [
        'Fruit'
    ],
    'Inflorescences':
    [
        'Inflorescences', 'Inflorescence', 'Sessile', 'Panicle', 'Flower head', 'Cyme', 'Glomerule', 
        'Fascicle', 'Umbel', 'Corymb', 'Rootlet', 'Spike', 'Dichasium', 'Fascicle',
        'Globose', 'Raceme', 'Fascicle', 'Umbel'
     ],
    'Sexuality':
    [
        'Sexuality', 'Axillary', 'Terminal'
    ],
    'Flower Colour':
    [
        'Flower colour', 'Flower color', 'Flower', 'Flowers'
    ],
    'Flower Shape':
    [
        'Flower shape', 'Petalled', 'Petal', 'Petals', 'Tubular', 'Apetal', 'Butterfly-shaped', 'Shaped', 'Flower', 'Flowers'
    ],
    'Sepal Shape':
    [
        'Sepal', 'Sepals', 'Connate'
    ],
    'Petal Shape':
    [
        'Petal', 'Petals', 'Tepals', 'Tepal', 'Tubular'
    ],
    'Aril Colour':
    [
        'Aril'
    ],
    'Seed Colour':
    [
        'Seed', 
    ]
}

with open('../../data/supportive/traits_Pierre_and_Andrei.json', 'w') as f:
    json.dump(traits_dict, f)
with open('../../data/supportive/colour_list.json', 'w') as f:
    json.dump(colors, f)

traits_list = list(traits_dict.keys())
traits_list += [trait.lower() for lst in list(traits_dict.values()) for trait in lst]

### Helper Functions

In [6]:
sw = list(stopwords.words('english'))
sw.append('like')
sw.append('color')
sw.append('colour')
sw.append('a')
sw.append('x')

In [7]:
def corresponding_keys(val, dictionary):
    """returns the corresponding key of a single value 
    assuming the values are lists.

    Args:
        val (string): string present in the dict
        dictionary (dict): dict with lists of stings as values

    Returns:
        list: list of matching keys
    """
    # Init list
    keys = []
    # Search the dict
    for k, v in dictionary.items():
        if val in v:
            keys.append(k)
    return keys
    
def jaccard_similarity(A, B):
    """Calculates the Jaccard similarity two sets.

    Args:
        A (Set): Set A
        B (Set): Set B

    Returns:
        Integer: 0.00 - 1.00
    """

    # Create sets just in case
    A = set(A)
    B = set(B)
    
    # Get intersection of two sets
    nominator = A.intersection(B)

    # Find union of two sets
    denominator = A.union(B)

    # Take the ratio of sizes
    similarity = len(nominator)/len(denominator)

    return similarity


def similarity(groundtruth, pred):
    """Calculates the normal similarity between two sets.

    Args:
        A (Set): Set A
        B (Set): Set B

    Returns:
        Integer: 0.00 - 1.00
    """

    # Create sets just in case
    groundtruth = set(groundtruth)
    pred = set(pred)

    # Find intersection of two sets
    nominator = groundtruth.intersection(pred)

    # Find union of two sets
    denominator = groundtruth

    # Take the ratio of sizes
    similarity = len(nominator)/(len(denominator))

    return similarity

def resentesize(lst):

    sentence = ''

    # Loop every 2 items 
    for item1, item2 in zip(lst[::2], lst[1::2]):
        
        # Set verb
        verb = 'has'
        chunk = item2.split(' ')[-1]
        if chunk not in nouns or chunk in colors:
            verb = 'is'

        sentence += f'{item1.capitalize()} {verb} {item2}. '
    
    return sentence


def get_wiki_main_image(title):
    url = 'https://en.wikipedia.org/w/api.php'
    data = {
        'action' :'query',
        'format' : 'json',
        'formatversion' : 2,
        'prop' : 'pageimages|pageterms',
        'piprop' : 'original',
        'titles' : title
    }
    response = requests.get(url, data)
    json_data = json.loads(response.text)
    return json_data['query']['pages'][0]['original']['source'] if len(json_data['query']['pages']) >0 else 'Not found'


## Retokize the Data

In [8]:
# Init empty dict for retok
species_datalist = collections.defaultdict(list)

root = "../../data/processed/"
file_name = root + 'Triples_Pierre.txt'

with open(file_name) as f:
   json_data = json.load(f)

for idx, (k, lst) in enumerate(json_data.items()):
    for l in lst:

        # Flatten list of lists
        flat_list = [item for sublist in l for item in sublist]
        # Retokens some sentence with multiple 'species'
        flat_list_multiple_sents = [list(v) for k, v in groupby(flat_list, lambda x: x != 'species') if k]
        
        for sentence in flat_list_multiple_sents:
            # Insert species
            sentence.insert(0, 'species')
            species_datalist[k].append(sentence)

file_name = root + 'Triples_Andrei.txt'

with open(file_name) as f:
   json_data = json.load(f)

for idx, (k, lst) in enumerate(json_data.items()):
    for l in lst:

        # Flatten list of lists
        flat_list = [item for sublist in l for item in sublist]
        # Retokens some sentence with multiple 'species'
        flat_list_multiple_sents = [list(v) for k, v in groupby(flat_list, lambda x: x != 'species') if k]
        
        for sentence in flat_list_multiple_sents:
            # Insert species
            sentence.insert(0, 'species')
            species_datalist[k].append(sentence)


## Match Data

In [9]:
def match_trait_against_DF(trait, df, which_df):
    """Return the corresponding traits

    Args:
        trait (string): The trait used
        df (DataFrame): The inserted DataFrame
        which_df (string): Which of the three DataFrames to use. 
                           Choices are ['Pierre', 'Andrei', 'Kissling']

    Raises:
        ValueError: If a DF other then ['Pierre', 'Andrei', 'Kissling']
                    is used.

    Returns:
        _type_: _description_
    """

    # Error
    df_choices = ['Pierre', 'Andrei', 'Kissling']
    if which_df not in df_choices:
        raise ValueError(f'Choose one of {df_choices}')

    column = ''
    if which_df == 'Pierre':
        if trait == 'Life Form':
            column = 'plant type'
        elif trait == 'Trunk':
            column = 'trunk and root'
        elif trait == 'Root':
            column = 'trunk and root'
        elif trait == 'Stem':
            column = 'stem shape'
        elif trait == 'Bark Colour':
            column = 'bark color'
        elif trait == 'Blade Colour':
            column = 'blade color'
        elif trait == 'Flower Colour':
            column = 'flower color'
        else:
            column = trait.lower()

    if which_df == 'Andrei':
        if trait == 'Phyllotaxis':
            column = ['Leaf position', 'Leaf upper side', 'Leaf lower side']
        elif trait == 'Spine':
            column = 'Thorns/spines'
        elif trait == 'Thornes':
            column = 'Thorns/spines'
        elif trait == 'Fruit':
            column = 'Fruit type'
        else:
            column = trait.lower()

            
    
    return column

In [10]:
species_traits = {}

for idx, (species, sentences) in enumerate(tqdm(species_datalist.items())):

    senteces_matches = collections.defaultdict(list)
    
    # if idx >= 1:
    #     continue

    for sentence in sentences:

        matches =  set(sentence) & set(traits_list)
        matches_color = set(colors) & set(sentence)
        if matches and not matches_color:
            #print(match, sentence)
            for match in matches:

                corresponding_traits = corresponding_keys(match.capitalize(), traits_dict)

                for corresponding_trait in corresponding_traits:
                    senteces_matches[corresponding_trait].append(sentence)

                    #print(corresponding_trait, sentence)

        elif matches_color and matches:
            for match in matches:

                corresponding_traits = corresponding_keys(match.capitalize(), traits_dict)
                for corresponding_trait in corresponding_traits:
                    if 'Colour' in corresponding_trait.split(' '):
                        senteces_matches[corresponding_trait].append(sentence)

                        #print("COLOR", corresponding_trait, sentence)

    species_traits[species] = senteces_matches

100%|██████████| 401/401 [00:00<00:00, 484.65it/s]


In [11]:
k = 5
google_forms_lst = []

for idx, species in enumerate(tqdm(species_traits.keys())):

    # if idx >= 1:
    #     continue

    df_name = ''
    df_select = [df_Andrei_dummies, df_Pierre]
    if species in df_Andrei_dummies.index:
        df_name = 'Andrei'
        df_select = df_select[0]
    elif species in df_Pierre.index:
        df_name = 'Pierre'
        df_select = df_select[1]
    # elif species in df_kissling:
    #     df_name = 'Kissling'
    #     df_select = df_select[2]
    else:
        print(species, 'Missing?', df_name)

    for trait in species_traits[species]:
        
        # Init 
        top_k_list = []
        # Match traits
        df_trait = match_trait_against_DF(trait, df_select, df_name)

        # Get present subtraits
        try:
            df_subset = df_select[df_select.index == species][df_trait]
            present_traits = df_subset.loc[:, df_subset.any()].columns

        # Skip traits without GT
        except:
            continue
        
        # Join items
        df_sent  = ' '.join(df_trait + present_traits)
        # Match against Sentences
        for lst in species_traits[species][trait]:
            sentence = ' '.join(lst)
            j_sim = jaccard_similarity(df_sent, sentence)
            # Append to list
            top_k_list.append((j_sim, lst))

        # Sort list and keep top K
        top_k_list.sort(reverse=True)
        top_sentences = []

        for (_, lst) in top_k_list:
            
            # Skip duplicated and remove jsim
            if lst not in top_sentences:
                top_sentences.append(lst)

        # Google Forms
        #google_forms_lst.append([species, trait, df_trait, present_traits.values])
        best_sentences = []

        # Extend lists below 5
        if len(top_sentences) < k:
            empties = [None] * (k - len(top_sentences))
            for empty in empties:
                top_sentences.append([empty])

        for i, top_sentence in enumerate(top_sentences):
            if i >= k:
                continue
            reconstructed_sent = resentesize(top_sentence)
            if reconstructed_sent:
                best_sentences.append(reconstructed_sent)
            else:
                best_sentences.append(np.NaN)

        #google_forms_lst.append(best_sentences)
        google_forms_lst.append((species, trait, df_trait, list(present_traits.values), *best_sentences))
    


100%|██████████| 401/401 [00:08<00:00, 49.76it/s]


In [12]:
# Google forms
df_google = pd.DataFrame(google_forms_lst, columns=['Species', 'Main Trait', 'GT Main Trait', 'GT Sub Traits', '1', '2', '3', '4', '5'])

In [13]:
df_google.to_csv(f'{root}top_sents_all_MainSubTraitsCombi.csv')

In [14]:
df_google_subset = df_google\
                    .dropna()\
                    .sample(n=20, axis=0, random_state=666)\

df_google_subset

,Species,Main Trait,GT Main Trait,GT Sub Traits,1,2,3,4,5
3347,Terminalia catappa,Fruit,fruit,"[Drupe, or drupe-like fruit]",Species has fruit. Fruit has fruit. Fruit has ...,Species is fruits. Fruits has fruit. Fruit has...,Species is fruits. Fruits has fruit. Fruit is ...,Species has fruit. Fruit has fruit. Fruit is c...,Species is fruits. Fruits has fruit. Fruit has...
697,Calotropis procera,Bark Colour,bark color,[Slice in light bark],Species has stem. Stem has bark. Bark has stem...,Species has root. Root has bark. Bark is root-...,Species is stems. Stems has stem. Stem has you...,Species has root. Root has bark. Bark has root...,Species has root. Root has bark. Bark is root-...
483,Blighia sapida,Flower Colour,flower color,"[Yellow coloured flower, Green coloured flower]",Species is buds. Buds has bud. Bud is into. In...,Species is flowers. Flowers has flower. Flower...,Species is flowers. Flowers has flower. Flower...,Species is flowers. Flowers has flower. Flower...,Species is flowers. Flowers has flower. Flower...
3123,Senna singueana,Stem,stem shape,[Circular stem section],Species has stem. Stem has stem. Stem has stra...,Species has stem. Stem has bark. Bark has stem...,Species has stem. Stem has bark. Bark has stem...,Species has stem. Stem has cutting. Cutting ha...,Species has stem. Stem has bark. Bark has stem...
952,Combretum collinum,Root,trunk and root,[Base of trunk straight],Species has root. Root has infusion. Infusion ...,Species has roots. Roots has root. Root is fre...,Species has root. Root has infusion. Infusion ...,Species has roots. Roots has root. Root of com...,Species has roots. Roots has root. Root is be....
770,Carica papaya,Root,trunk and root,[Base of trunk straight],Species has trunk. Trunk has trunk. Trunk has ...,Species has root. Root has root. Root has poke...,Species has root. Root has juice. Juice has ro...,Species has root. Root has phloem. Phloem has ...,Species has root. Root is sliver. Sliver is ro...
191,Adenium obesum,Life Form,plant type,"[Tree, Shrub]",Species is flowers. Flowers has structure. Str...,Species is plants. Plants has plant. Plant has...,Species is flowers. Flowers has shrub. Shrub h...,Species has inflorescence. Inflorescence has s...,Species has trunk. Trunk has trunk. Trunk is i...
2194,Landolphia heudelotii,Fruit,fruit,"[Berry, or berry-like fruit]",Species is fruits. Fruits has fruit. Fruit is ...,Species has fruit. Fruit is e.. E. is ci-e.. C...,Species has fruit. Fruit has fruit. Fruit has ...,Species has fruit. Fruit has s. S has fruit s....,Species has fruit. Fruit has p. P has fruit p....
244,Albizia adianthifolia,Fruit,fruit,[Pod],Species has fruit. Fruit has fruit. Fruit has ...,Species has fruit. Fruit has fruit. Fruit has ...,Species is fruits. Fruits has fruit. Fruit is ...,Species is seeds. Seeds has seed. Seed has rat...,Species has fruit. Fruit is extract. Extract i...
1923,Grewia lasiodiscus,Fruit,fruit,"[Drupe, or drupe-like fruit, Berry, or berry-l...",Species has fruit. Fruit has tree. Tree has fr...,Species has fruit. Fruit has fruit. Fruit has ...,Species has fruit. Fruit has fruit. Fruit is r...,Species is fruits. Fruits has fruit. Fruit has...,Species has fruit. Fruit has fruit. Fruit has ...


In [15]:
random_sample_species = df_google_subset['Species'].values

images_links = {}
for species in tqdm(random_sample_species):
    species_ = species.replace(' ', '_')
    try:
        img_url = get_wiki_main_image(species_)
        if img_url[-3:] != 'jpg':
            img_url = 'https://upload.wikimedia.org/wikipedia/commons/1/14/No_Image_Available.jpg'
        # if img_url[-3:].isupper():
        #     img_url = 'https://upload.wikimedia.org/wikipedia/commons/1/14/No_Image_Available.jpg'
        images_links[species] = img_url
    except:
        images_links[species] = 'https://upload.wikimedia.org/wikipedia/commons/1/14/No_Image_Available.jpg'

# df_google_subset = df_google[df_google['Species'].isin(random_sample_species)]
df_google_subset = df_google_subset.set_index('Species')
df_google_subset["URL"] = pd.Series(images_links)
df_google_subset.to_csv(f'{root}top_sents_all_MainSubTraitsCombi_Random20Subset.csv', sep='\t')#, quoting=csv.QUOTE_ALL)

100%|██████████| 20/20 [00:03<00:00,  5.51it/s]


In [16]:
df_google_subset

,Main Trait,GT Main Trait,GT Sub Traits,1,2,3,4,5,URL
Species,,,,,,,,,
Terminalia catappa,Fruit,fruit,"[Drupe, or drupe-like fruit]",Species has fruit. Fruit has fruit. Fruit has ...,Species is fruits. Fruits has fruit. Fruit has...,Species is fruits. Fruits has fruit. Fruit is ...,Species has fruit. Fruit has fruit. Fruit is c...,Species is fruits. Fruits has fruit. Fruit has...,https://upload.wikimedia.org/wikipedia/commons...
Calotropis procera,Bark Colour,bark color,[Slice in light bark],Species has stem. Stem has bark. Bark has stem...,Species has root. Root has bark. Bark is root-...,Species is stems. Stems has stem. Stem has you...,Species has root. Root has bark. Bark has root...,Species has root. Root has bark. Bark is root-...,https://upload.wikimedia.org/wikipedia/commons...
Blighia sapida,Flower Colour,flower color,"[Yellow coloured flower, Green coloured flower]",Species is buds. Buds has bud. Bud is into. In...,Species is flowers. Flowers has flower. Flower...,Species is flowers. Flowers has flower. Flower...,Species is flowers. Flowers has flower. Flower...,Species is flowers. Flowers has flower. Flower...,https://upload.wikimedia.org/wikipedia/commons...
Senna singueana,Stem,stem shape,[Circular stem section],Species has stem. Stem has stem. Stem has stra...,Species has stem. Stem has bark. Bark has stem...,Species has stem. Stem has bark. Bark has stem...,Species has stem. Stem has cutting. Cutting ha...,Species has stem. Stem has bark. Bark has stem...,https://upload.wikimedia.org/wikipedia/commons...
Combretum collinum,Root,trunk and root,[Base of trunk straight],Species has root. Root has infusion. Infusion ...,Species has roots. Roots has root. Root is fre...,Species has root. Root has infusion. Infusion ...,Species has roots. Roots has root. Root of com...,Species has roots. Roots has root. Root is be....,https://upload.wikimedia.org/wikipedia/commons...
Carica papaya,Root,trunk and root,[Base of trunk straight],Species has trunk. Trunk has trunk. Trunk has ...,Species has root. Root has root. Root has poke...,Species has root. Root has juice. Juice has ro...,Species has root. Root has phloem. Phloem has ...,Species has root. Root is sliver. Sliver is ro...,https://upload.wikimedia.org/wikipedia/commons...
Adenium obesum,Life Form,plant type,"[Tree, Shrub]",Species is flowers. Flowers has structure. Str...,Species is plants. Plants has plant. Plant has...,Species is flowers. Flowers has shrub. Shrub h...,Species has inflorescence. Inflorescence has s...,Species has trunk. Trunk has trunk. Trunk is i...,https://upload.wikimedia.org/wikipedia/commons...
Landolphia heudelotii,Fruit,fruit,"[Berry, or berry-like fruit]",Species is fruits. Fruits has fruit. Fruit is ...,Species has fruit. Fruit is e.. E. is ci-e.. C...,Species has fruit. Fruit has fruit. Fruit has ...,Species has fruit. Fruit has s. S has fruit s....,Species has fruit. Fruit has p. P has fruit p....,https://upload.wikimedia.org/wikipedia/commons...
Albizia adianthifolia,Fruit,fruit,[Pod],Species has fruit. Fruit has fruit. Fruit has ...,Species has fruit. Fruit has fruit. Fruit has ...,Species is fruits. Fruits has fruit. Fruit is ...,Species is seeds. Seeds has seed. Seed has rat...,Species has fruit. Fruit is extract. Extract i...,https://upload.wikimedia.org/wikipedia/commons...
